In [1]:
# imports

import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import datetime
import itertools
import random
import time

import pmdarima as pm

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, RepeatVector, TimeDistributed

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
'''
This method is specific function for this utility. It collects the data and preprocesses it.
'''
def get_data():
    
    # loading the data from csv files.
    nestle_stock_data = pd.read_csv('NSRGY.csv')
    danone_stock_data = pd.read_csv('DANOY.csv')
    milk_price = pd.read_csv('Class_III_Milk.csv')
    
    # bringing date into correct date type
    nestle_stock_data['Date'] =  pd.to_datetime(nestle_stock_data['Date'], format='%Y-%m-%d')
    danone_stock_data['Date'] =  pd.to_datetime(danone_stock_data['Date'], format='%Y-%m-%d')
    milk_price['Date'] =  pd.to_datetime(milk_price['Date'], format='%b %d, %Y')
    
    valid_dates = nestle_stock_data['Date'].tolist()
    
    # Converting the volume column of class III milk price into numerical data type
    milk_price['Vol.'] = milk_price['Vol.'].str.replace('K', '')
    milk_price['Vol.'] = milk_price['Vol.'].astype('float64')
    milk_price['Vol.'] = milk_price['Vol.'] * 1000
    
    milk_price = milk_price.iloc[::-1]
    
    # Creating farmers data
    volume_of_milk_sold_in_lbs = [random.randint(5000,11000) for _ in range(5000*len(valid_dates))]
    price_received_per_lbs = [random.randint(1250, 1750)/100 for _ in range(5000*len(valid_dates))]
    date_list_users = valid_dates * 5000
    farmer_ids = list(itertools.chain.from_iterable(itertools.repeat(x, len(valid_dates)) for x in range(1,5001)))
    
    users = pd.DataFrame(list(zip(farmer_ids, date_list_users, volume_of_milk_sold_in_lbs, price_received_per_lbs)), 
                         columns=['farmer_id', 'date', 'volume_of_milk_sold_by_farmers', 'milk_price_received_by_farmers']) 
    
    milk_price.set_index('Date', inplace=True)
    nestle_stock_data.set_index('Date', inplace= True)
    danone_stock_data.set_index('Date',inplace=True)
    
    return nestle_stock_data, danone_stock_data, milk_price, users

In [3]:
'''
Converts historical data into LSTM expected inputs and outputs formats
'''
def input_formater(model_type, data, n_input, n_out=21):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    data --> input data for formatting
    n_input --> number of prior inputs used
    n_out --> number of predictions we are going to make (By default = 21, average business days in a month)
    '''
    
    X, y = list(), list()
    in_start = 0
    
    if model_type == 'simple' or model_type == 'enc-dec-univar':
        
        # step over the entire history one time step at a time
        for _ in range(len(data)):
            # define the end of the input sequence
            in_end = in_start + n_input
            out_end = in_end + n_out

            # ensure we have enough data for this instance
            if out_end < len(data):
                x_input = data[in_start:in_end, 0]
                x_input = x_input.reshape((len(x_input), 1))
                X.append(x_input)
                y.append(data[in_end:out_end, 0])
            # move along one time step
            in_start += 1
            
    elif model_type == 'enc-dec-multivar':

        # step over the entire history one time step at a time
        for _ in range(len(data)):
            # define the end of the input sequence
            in_end = in_start + n_input
            out_end = in_end + n_out
            # ensure we have enough data for this instance
            if out_end < len(data):
                X.append(data[in_start:in_end, :])
                y.append(data[in_end:out_end, 0])
            # move along one time step
            in_start += 1
        
    return array(X), array(y)

In [4]:
'''
This is where we build models. 
Future scope includes more model tuning features.
'''
def build_model(model_type, train, n_input, n_out, verbose, epochs, batch_size):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    train --> training data
    n_input --> number of prior inputs used
    n_out --> number of predictions we are going to make (By default = 21, average business days in a month)
    verbose --> whether we want to display results on each fit
    eopchs --> number of epochs
    batch_size --> batch size
    '''
    # prepare data in proper format
    # LSTM expected input = [samples, timesteps, features]
    train_x, train_y = input_formater(model_type, train, n_input, n_out)
    
    # define parameters
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    
    # define model
    if model_type == 'simple':
        model = Sequential()
        model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(n_outputs))
        model.compile(loss='mse', optimizer='adam')
    
    # define model
    if model_type == 'enc-dec-univar' or model_type == 'enc-dec-multivar':
        
        # reshape output into [samples, timesteps, features]
        train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
        
        model = Sequential()
        model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
        model.add(RepeatVector(n_outputs))
        model.add(LSTM(200, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(100, activation='relu')))
        model.add(TimeDistributed(Dense(1)))
        model.compile(loss='mse', optimizer='adam')
    
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

In [5]:
'''
This method gives us the forecast for LSTM models.
'''
def forecast(model_type, model, history, n_input):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    model --> trained model
    history --> training data 
    n_input --> number of prior inputs used
    '''
    data = array(history)
    
    if model_type == 'simple' or model_type == 'enc-dec-univar':
        
        # retrieve last observations for input data
        input_x = data[-n_input:, 0]
        # reshape into [1, n_input, 1]
        input_x = input_x.reshape((1, len(input_x), 1))
    
    elif model_type == 'enc-dec-multivar':
    
        # retrieve last observations for input data
        input_x = data[-n_input:, :]
        # reshape into [1, n_input, n]
        input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
    
    # forecast the next day
    yhat = model.predict(input_x, verbose=0)

    # we only want the vector forecast
    yhat = yhat[0]
    return yhat

In [6]:
'''
This method evaluates forecasts against expected values.
Different evaluation metrics can be implemented.
'''
def evaluate_forecasts_rmse(model_type, actual, predicted):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    actual --> ground truth
    predicted --> predicted results
    '''
    if model_type == 'simple':
        diffs = actual - predicted
        rmse = np.sqrt(np.mean(diffs**2))

    elif model_type == 'enc-dec-univar':
        diffs = actual - predicted[:,0,0]
        rmse = np.sqrt(np.mean(diffs**2))
    
    # desired time series should be at 0th index.
    elif model_type == 'enc-dec-multivar':
        diffs = actual[:,0] - predicted[:,0,0]
        rmse = np.sqrt(np.mean(diffs**2))
        
    return rmse

In [7]:
'''
This method evaluates the overall model.
'''
def evaluate_model(model_type, train, test, n_input, n_out, verbose, epochs, batch_size):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    train --> training data
    test --> validation data
    n_input --> number of prior inputs used
    n_out --> number of predictions we are going to make (By default = 21, average business days in a month)
    verbose --> whether we want to display results on each fit
    eopchs --> number of epochs
    batch_size --> batch size
    '''
    # fit model
    model = build_model(model_type, train, n_input, n_out, verbose, epochs, batch_size)
    
    # history is a list of daily data
    history = [x for x in train]
    
    # walk-forward validation over each day
    predictions = list()
    
    for i in range(len(test)):
        # predict the day
        yhat_sequence = forecast(model_type, model, history, n_input)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next day
        history.append(test[i, :])
    
    # evaluate prediction for each day
    predictions = array(predictions)
    rmse = evaluate_forecasts_rmse(model_type, test, predictions)
    return rmse, model

In [8]:
'''
This method is for training LSTM model with different parameters.
'''
def call_LSTM_model(model_type, train, test, n_inputs, n_out, plot_rmse, verbose, epochs, batch_size):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    train --> training data
    test --> validation data
    n_input --> number of prior inputs used
    n_out --> number of predictions we are going to make (By default = 21, average business days in a month)
    plot_rmse --> whether you want to plot the rmse vs number of prior inputs considered graph.
    verbose --> whether we want to display results on each fit
    eopchs --> number of epochs
    batch_size --> batch size
    '''
    n_input_rmses = []
    best_rmse = float('inf')
    best_model = Sequential()
    
    # model tuning --> selecting best n_inputs
    for n_input in range(1, n_inputs):
        rmse, model = evaluate_model(model_type, train, test, n_input, n_out, verbose, epochs, batch_size)

        if rmse < best_rmse:
            best_model = model
            best_rmse = rmse

        n_input_rmses.append(rmse)
    
    best_n_input = n_input_rmses.index(min(n_input_rmses))+1
    
    if plot_rmse:
        plt.figure(figsize = (20,10))
        plt.plot(range(1,n_inputs), n_input_rmses)
        plt.title('RMSE for models with different number of prior days.')
        plt.ylabel('RMSE')
        plt.xlabel('Number of prior days considered for model')
        plt.show()
    
    return best_model, best_n_input

In [9]:
'''
This method is for vanilla LSTM and Encoder-Decoder Univariate LSTM models.
'''
def predict_LSTM_model(model_type, milk_price):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    milk_price --> class III milk price data
    '''
    if model_type == 'enc-dec-univar' or 'simple':
        
        scaler = MinMaxScaler(feature_range=(0, 1))
        only_milk_price = milk_price.loc[:, ['Price']]
        only_milk_price_array = scaler.fit_transform(only_milk_price)

        train = only_milk_price_array[:211]
        test = only_milk_price_array[211:]

        best_model, best_n_input = call_LSTM_model(model_type, train, test, 25, 21, False, 0, 20, 16)

        pred_vanilla = forecast(model_type, best_model, only_milk_price_array, best_n_input)
        pred_vanilla = pred_vanilla.reshape((-1, 1))
        pred_vanilla = scaler.inverse_transform(pred_vanilla)

        return pred_vanilla.mean()

In [10]:
'''
This method is for Encoder-Decoder Multivariate LSTM models.
'''
def predict_LSTM_multivar_model(model_type, nestle_stock_data, danone_stock_data, milk_price, users):
    '''
    model_type --> Type of model: simple(i.e. vanilla) or Encoder-Decoder Univariate or Multivariate
    rest --> input data
    '''    
    if model_type == 'enc-dec-multivar':
        
        combined = []
        users_summary = []

        users_summary = users.groupby('date')['volume_of_milk_sold_by_farmers',
                                              'milk_price_received_by_farmers'].mean()

        combined = pd.concat([users_summary, milk_price.loc[:,['Price','Vol.']]], axis=1)
        combined = combined.rename(columns={"Price": "class_III_milk_price", 
                                            "Vol.": "class_III_milk_volume"})
        
        combined = pd.concat([combined, nestle_stock_data.loc[:,['Close','Volume']]], axis=1)
        combined = combined.rename(columns={"Close": "nestle_stock_close_price", 
                                            "Volume": "nestle_stock_volume"})
       
        combined = pd.concat([combined, danone_stock_data.loc[:,['Close','Volume']]], axis=1)
        combined = combined.rename(columns={"Close": "danone_stock_close_price", 
                                            "Volume": "danone_stock_volume"})
       
        # bringing class III milk price at 0th index
        combined = combined.loc[:, ['class_III_milk_price','class_III_milk_volume', 
                           'volume_of_milk_sold_by_farmers','milk_price_received_by_farmers',
                           'nestle_stock_close_price', 'nestle_stock_volume', 
                           'danone_stock_close_price', 'danone_stock_volume']]
        
        combined = combined.values
        
        train = combined[:211]
        test = combined[211:]
        
        best_model, best_n_input = call_LSTM_model(model_type, train, test, 10, 21, False, 0, 20, 16)
        
        pred_vanilla = forecast(model_type, best_model, combined, best_n_input)
        pred_vanilla = pred_vanilla.reshape((-1, 1))
        return pred_vanilla.mean()

In [11]:
'''
This method is for training Univariate Auto ARIMA model.
'''
def AutoArimaModelTrain(trainData, validationData, timeSeriesFeature, frequency, predPeriod, plot_arima):
    '''
        trainData --> training data
        validationData --> validation data
        timeSeriesFeature --> univariate time series feature selected for the analysis
        frequency --> frequency of the series (the value of m)
        predPeriod --> number of future prediction points = number of rows in validation data
        plot_arima --> whether to plot auto arima model with prediction
    '''
    ARIMAmodel = pm.arima.auto_arima(trainData[timeSeriesFeature], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=5, max_q=5, # maximum p and q
                      m=frequency,      # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   
                      start_P=0, 
                      D=1,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
    
    ARIMAmodel.fit(trainData[timeSeriesFeature])
    
    # number of future prediction points = number of rows in validation data 
    future_forecast = ARIMAmodel.predict(n_periods=predPeriod)
    future_forecast = pd.DataFrame(future_forecast,index = validationData.index,columns=['Prediction'])
    
    if plot_arima:
        plt.figure(figsize = (20,10))
        plt.plot(trainData[timeSeriesFeature], label = 'Training Data', color = 'green')
        plt.plot(validationData[timeSeriesFeature], label = 'Validation Data', color = 'blue')
        plt.plot(future_forecast['Prediction'], label = 'Prediction', color = 'orange')
        plt.title(timeSeriesFeature)
        plt.legend()
        plt.show()
    
    error_model = pd.concat([validationData[timeSeriesFeature],future_forecast],axis=1)
    error_model = error_model.diff(axis=1)
    error_model['Pred_err_abs'] = error_model.loc[:, ['Prediction']].abs()
    
    # Absolute Error
    modelAbsErr = error_model['Pred_err_abs'].sum()
    
    return modelAbsErr, frequency
    

In [12]:
'''
This method is for predicting the future prices based best auto ARIMA model.
'''
def AutoARMIAPredict(trainData, timeSeriesFeature, frequency, predPeriod):
    '''
        trainData --> training data
        validationData --> validation data
        timeSeriesFeature --> univariate time series feature selected for the analysis
        frequency --> frequency of the series (the value of m)
        predPeriod = number of future prediction points = number of rows in validation data
    '''
    ARIMAmodel = pm.arima.auto_arima(trainData[timeSeriesFeature], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=5, max_q=5, # maximum p and q
                      m=frequency,      # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   
                      start_P=0, 
                      D=1,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
    
    ARIMAmodel.fit(trainData[timeSeriesFeature])
    
    # number of future prediction points = number of rows in validation data 
    predictions = ARIMAmodel.predict(n_periods=predPeriod)
    return predictions.mean()

In [13]:
'''
This method is for ARIMA models.
'''
def predict_ARIMA_model(milk_price, n_out):
    '''
    milk_price --> class III milk price data
    n_out --> number of predictions we are going to make (By default = 21, average business days in a month)
    '''
    least_error = float('inf')
    best_freq = 1
    abs_errors = []
    
    train = milk_price[:211]
    test = milk_price[211:]
    
    freq_list = [1,12,52]
    
    for i in freq_list:
        error, freq = AutoArimaModelTrain(train, test, 'Price', i, test.shape[0], False)
        
        abs_errors.append(error)
        
        if error < least_error:
            best_freq = i
            least_error = error
    
    return AutoARMIAPredict(milk_price, 'Price', best_freq, n_out)

In [14]:
'''
The prediction utility
'''
def Prediction_Utility():
    
    nestle_stock_data, danone_stock_data, milk_price, users = get_data()
    
    ARIMA_pred = predict_ARIMA_model(milk_price, 21)
    
    print(f'ARIMA model prediction is {ARIMA_pred}')
    
    vanilla_pred = predict_LSTM_model('simple', milk_price)
    
    print(f'Simple LSTM model prediction is {vanilla_pred}')
    
    enc_dec_LSTM_pred = predict_LSTM_model('enc-dec-univar', milk_price)
    
    print(f'Enc Dec LSTM model prediction is {enc_dec_LSTM_pred}')
    
    #enc_dec_mul_LSTM_pred = predict_LSTM_multivar_model(
    #    'enc-dec-multivar',nestle_stock_data, danone_stock_data, milk_price, users)
    
    #print(f'Enc Dec Multi LSTM model prediction is {enc_dec_mul_LSTM_pred}')
    
    
    return (ARIMA_pred + vanilla_pred + enc_dec_LSTM_pred)/3

In [15]:
prediction_amount = Prediction_Utility()
print(f'prediction amount for next month is {prediction_amount}')

ARIMA model prediction is 15.9815994417871
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Simple LSTM model prediction is 15.656394004821777
Enc Dec LSTM model prediction is 15.67132568359375
prediction amount for next month is 15.769773043400875


--------

This is average predicted class III milk price based on the data. Now let's compare this with next month's future class III milk quote.

From CME group website, Future Quote for class III milk in May is '$16.25'

The prediction of this utility about the class III milk price in May is '$15.77'

#### Therfore, Farmers should go short!

#### Strategy:
If prediction for next month is less than the future quote.

    The farmer should sell next month class III milk futures. (should go short)
    or The farmer should purchase PUT Options. check the premium of that month. 

If the prediction for given month is more than the future quote.

    The farmer can take advantage of this higher price and sell his milk at higher price. (should not go short on that months class III milk futures.)
    or purchase PUT Options again with premium check.